# CJE Planning: Optimize Your Evaluation Budget

Before running a large evaluation, answer these questions:

- **How many samples** do I need?
- **How many oracle labels** are worth the cost?
- **What effect size** can I reliably detect?

CJE's planning tools help you find the optimal allocation between cheap surrogate scores and expensive oracle labels.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cimo-labs/cje/blob/main/examples/cje_planning.ipynb)

In [ ]:
# Install CJE
!pip install -q --upgrade cje-eval

## The Planning Problem

CJE calibrates cheap judge scores to match expensive oracle outcomes. The variance of your estimates depends on two components:

$$\text{Var}(\hat{\theta}) = \frac{\sigma^2_{\text{eval}}}{n} + \frac{\sigma^2_{\text{cal}}}{m}$$

Where:
- **n** = total samples (evaluated by cheap judge)
- **m** = oracle-labeled samples (subset of n)
- **σ²_eval** = evaluation variance (inherent noise in policy performance)
- **σ²_cal** = calibration variance (uncertainty in learning judge→oracle mapping)

**The tradeoff:** More oracle labels (↑m) reduces calibration uncertainty but costs more. CJE finds the optimal balance.

## 1. Load Pilot Data

Planning requires **pilot data** — a small sample to estimate the variance components. You can use:

1. **Your own pilot study** (recommended): 100-500 samples with oracle labels
2. **Arena sample data** (below): Real-world reference from Chatbot Arena

The pilot data should be representative of your actual evaluation setting.

In [ ]:
# Download Arena sample data as reference
import urllib.request
from pathlib import Path

DATA_DIR = Path("arena_sample")
if not (DATA_DIR / "fresh_draws" / "base_responses.jsonl").exists():
    print("Downloading sample data...")
    DATA_DIR.mkdir(exist_ok=True)
    (DATA_DIR / "fresh_draws").mkdir(exist_ok=True)
    
    BASE_URL = "https://raw.githubusercontent.com/cimo-labs/cje/main/examples/arena_sample"
    for f in ["base_responses.jsonl", "clone_responses.jsonl", 
              "parallel_universe_prompt_responses.jsonl", "unhelpful_responses.jsonl"]:
        urllib.request.urlretrieve(f"{BASE_URL}/fresh_draws/{f}", DATA_DIR / "fresh_draws" / f)
    print("Done!")
else:
    print(f"Data exists at {DATA_DIR.absolute()}")

## 2. Fit Variance Model

The variance model learns σ²_eval and σ²_cal from your pilot data by measuring estimate variance at different (n, m) allocations.

**Important:** Use a **single policy** for fitting. The model assumes a consistent variance structure — mixing policies with different characteristics (e.g., base vs adversarial) violates this assumption and causes poor fit.

In [ ]:
from cje import fit_variance_model
from cje.data.fresh_draws import load_fresh_draws_auto

# Load pilot data - use a SINGLE policy for variance model fitting
# The 1/n + 1/m model assumes consistent variance structure
base_data = load_fresh_draws_auto("arena_sample/fresh_draws", "base")

print(f"Loaded base policy: {len(base_data.samples)} samples")
print(f"Oracle labels: {sum(1 for s in base_data.samples if s.oracle_label is not None)}")

In [ ]:
# Fit the variance model using single policy (takes ~30 seconds)
# Using multiple policies violates the model assumptions and causes poor fit
model = fit_variance_model({"base": base_data}, verbose=True)

# R² > 0.85 indicates the 1/n + 1/m model fits well
if model.r_squared < 0.5:
    print("\n⚠️  Low R² suggests the variance model may not fit your data well.")
    print("   Check that oracle labels are randomly sampled within your pilot.")

## 3. Specify Your Costs

**Critical:** Use actual dollar costs per API call. This ensures budget is in real dollars.

| Surrogate (Judge) | Oracle | Cost Ratio |
|-------------------|--------|------------|
| GPT-4o-mini ($0.01) | GPT-4o ($0.16) | 16× |
| Claude Haiku ($0.008) | Claude Sonnet ($0.09) | 11× |
| Llama-70B ($0.002) | Human ($2.00) | 1000× |

In [ ]:
from cje import CostModel

# Example: GPT-4o-mini as judge, GPT-4o as oracle
cost_model = CostModel(
    surrogate_cost=0.01,  # $/call for judge
    oracle_cost=0.16      # $/call for oracle
)

print(f"Surrogate cost: ${cost_model.surrogate_cost}/call")
print(f"Oracle cost: ${cost_model.oracle_cost}/call")
print(f"Cost ratio: {cost_model.oracle_cost/cost_model.surrogate_cost:.0f}×")

## 4. Budget-Constrained Planning

**Question:** "I have $X budget — what effect size can I reliably detect?"

In [ ]:
from cje import plan_evaluation

# Plan with $5,000 budget
plan = plan_evaluation(
    budget=5000,
    variance_model=model,
    cost_model=cost_model
)

print(plan.summary())

In [ ]:
# Interpret the results
print(f"📊 What this means:")
print(f"")
print(f"   1. Collect {plan.n_samples:,} responses, each scored by your surrogate judge")
print(f"   2. Randomly label {plan.m_oracle} of them with oracle ({plan.oracle_fraction:.1%} of samples)")
print(f"   3. Run CJE to get calibrated estimates")
print(f"")
print(f"   With this setup, you can detect a {plan.mde:.1%} difference between policies")
print(f"   with 80% statistical power (α=0.05).")

## 5. MDE-Constrained Planning

**Question:** "I need to detect X% differences — what will it cost?"

In [ ]:
from cje import plan_for_mde

# Plan to detect 2% differences
plan_2pct = plan_for_mde(
    target_mde=0.02,
    variance_model=model,
    cost_model=cost_model
)

print(f"To detect 2% differences with 80% power:")
print(f"  Budget needed: ${plan_2pct.total_cost:,.0f}")
print(f"  Samples: {plan_2pct.n_samples:,} total, {plan_2pct.m_oracle} oracle labels")

In [ ]:
# Compare different MDE targets
print(f"📊 MDE vs Budget Tradeoff")
print(f"="*40)
print(f"{'MDE':<10} {'Budget':<15} {'Samples':<10} {'Oracle'}")
print(f"-"*40)

for target in [0.05, 0.03, 0.02, 0.015, 0.01]:
    p = plan_for_mde(target_mde=target, variance_model=model, cost_model=cost_model)
    print(f"{target:.1%}       ${p.total_cost:>10,.0f}    {p.n_samples:>7,}   {p.m_oracle:>5}")

## 6. Visualize Tradeoffs

The planning dashboard shows three views:

1. **MDE vs Budget** — How precision improves with spending
2. **Power Curve** — Probability of detecting different effect sizes
3. **Cost Sensitivity** — How oracle cost affects the tradeoff

In [ ]:
from cje.visualization import plot_planning_dashboard
import matplotlib.pyplot as plt

fig = plot_planning_dashboard(model, cost_model)
plt.show()

## 7. The Planning Loop

Planning is iterative. Start with one constraint and check if the other is acceptable:

```python
# "I have $10K but need 1.5% MDE"
plan = plan_evaluation(budget=10000, ...)  # → MDE = 2.1% (too high!)
plan = plan_for_mde(target_mde=0.015, ...) # → $18K (too expensive!)
plan = plan_for_mde(target_mde=0.018, ...) # → $12K (compromise?)
```

**Rule of thumb:** Target MDE should be 2-3× smaller than differences you actually care about. If you care about 5% differences, aim for 2% MDE.

In [ ]:
# Interactive planning example
print("Example planning session:")
print("="*50)

# Start with budget constraint
initial = plan_evaluation(budget=10000, variance_model=model, cost_model=cost_model)
print(f"\n1. With $10K budget: MDE = {initial.mde:.1%}")

# Check what it costs to hit a tighter MDE
target = plan_for_mde(target_mde=0.015, variance_model=model, cost_model=cost_model)
print(f"2. To hit 1.5% MDE: need ${target.total_cost:,.0f}")

# Find a compromise
compromise = plan_for_mde(target_mde=0.02, variance_model=model, cost_model=cost_model)
print(f"3. Compromise at 2% MDE: ${compromise.total_cost:,.0f}")

## Summary

### Quick Reference

```python
from cje import fit_variance_model, CostModel, plan_evaluation, plan_for_mde
from cje.data.fresh_draws import load_fresh_draws_auto

# 1. Fit variance model from pilot data (use SINGLE policy)
base_data = load_fresh_draws_auto("pilot_dir", "base")
model = fit_variance_model({"base": base_data})

# 2. Specify costs (use actual $/call)
cost_model = CostModel(surrogate_cost=0.01, oracle_cost=0.16)

# 3a. Budget → MDE
plan = plan_evaluation(budget=5000, variance_model=model, cost_model=cost_model)

# 3b. MDE → Budget  
plan = plan_for_mde(target_mde=0.02, variance_model=model, cost_model=cost_model)
```

### Next Steps

Once you have your plan, run the evaluation:

[![Core Tutorial](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cimo-labs/cje/blob/main/examples/cje_core_demo.ipynb)

**Documentation:**
- [CJE GitHub](https://github.com/cimo-labs/cje)
- [Blog: Why Your Metrics Lie](https://cimolabs.com/blog/metrics-lying)